In [6]:
import os
import re 
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import base64

In [8]:

def split_and_encrypt(input_path, chunk_size, base_password):
    base, ext = os.path.splitext(input_path)
    part_num = 1
    
    with open(input_path, 'rb') as infile:
        while True:
            chunk = infile.read(chunk_size)
            if not chunk:
                break
            
            # Generar clave única para esta parte (base_password + part_num)
            salt = os.urandom(16)
            kdf = PBKDF2HMAC(
                algorithm=hashes.SHA256(),
                length=32,
                salt=salt,
                iterations=100000,
            )
            key = base64.urlsafe_b64encode(kdf.derive(f"{base_password}{part_num}".encode()))
            cipher = Fernet(key)
            encrypted_chunk = cipher.encrypt(chunk)
            part_name = f"{base}_part{part_num}{ext}.enc"
            with open(part_name, 'wb') as outfile:
                outfile.write(salt + encrypted_chunk)
            
            part_num += 1
    
    print(f"Archivo dividido y cifrado en {part_num-1} partes.")

In [10]:
def decrypt_and_merge(first_part_path, output_path, base_password):
    dir_path = os.path.dirname(first_part_path) or '.'  # Usa '.' si la ruta es vacía
    base_ext = os.path.basename(first_part_path)
    
    # Extraer nombre base, número de parte y extensión
    match = re.match(r'^(.*?)_part(\d+)(\..+\.enc)$', base_ext)
    if not match:
        raise ValueError("Formato incorrecto. Ejemplo esperado: nombre_part1.zip.enc")
    base_name = match.group(1)
    ext = match.group(3)
    
    # Buscar todas las partes en el directorio correcto
    part_files = []
    for f in os.listdir(dir_path):  # Ahora dir_path es '.' si está vacío
        full_path = os.path.join(dir_path, f)
        if os.path.isfile(full_path) and f.startswith(f"{base_name}_part") and f.endswith(ext):
            part_num = int(re.search(r'_part(\d+)' + re.escape(ext) + '$', f).group(1))
            part_files.append((part_num, full_path))
    
    # Verificar partes secuenciales
    part_files.sort()
    expected_parts = list(range(1, part_files[-1][0] + 1)) if part_files else []
    missing = [p for p in expected_parts if p not in [pf[0] for pf in part_files]]
    if missing:
        raise ValueError(f"Partes faltantes: {missing}")
    
    # Descifrar y unir
    with open(output_path, 'wb') as outfile:
        for part_num, part_path in part_files:
            with open(part_path, 'rb') as infile:
                data = infile.read()
                salt, encrypted_chunk = data[:16], data[16:]
                
                # Regenerar clave
                kdf = PBKDF2HMAC(
                    algorithm=hashes.SHA256(),
                    length=32,
                    salt=salt,
                    iterations=100000,
                )
                key = base64.urlsafe_b64encode(kdf.derive(f"{base_password}{part_num}".encode()))
                cipher = Fernet(key)
                
                # Descifrar
                decrypted_chunk = cipher.decrypt(encrypted_chunk)
                outfile.write(decrypted_chunk)
    for _, part_path in part_files:
            os.remove(part_path)
    print(f"Archivo fusionado: {output_path}")

In [17]:
split_and_encrypt("prueba.pdf.enc.zip", 1 * 1024 * 1024, "mi_contraseña_secreta")

Archivo dividido y cifrado en 13 partes.


In [15]:
decrypt_and_merge("prueba.pdf.enc_part1.zip.enc", "Reporte.zip", "mi_contraseña_secreta")

Archivo fusionado: Reporte.zip
